In [1]:
import yaml
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

from src.utils import *

plt.style.use('seaborn')

with open("config.yaml") as f:
    config = yaml.safe_load(f)
    
def is_metal_artist(x):
    if not x:
        return
    else:
        return any(g in metal_genres for g in x)

In [2]:
# Use the SpotifyClientCredentials object to authenticate with the Spotify API
client_credentials_manager = SpotifyClientCredentials(
    client_id=config.get('client_id'), 
    client_secret=config.get('client_secret'),
)

sp = spotipy.Spotify(
    client_credentials_manager = SpotifyClientCredentials(
        client_id=config.get('client_id'), 
        client_secret=config.get('client_secret'),
    )
)

In [ ]:
for l in config['alpha_num'][18:19]:
    
    df = pd.read_csv(f'data/raw/bands_{l}.csv')
    
    max_iter = 5
    artist_dict = []
    no_data_artist = []

    for name in tqdm(set(df.name.tolist()), f'{l} data'):

        total_artist = len(artist_dict)

        artist = (
            sp.search(name, type='artist')
            ['artists']
            ['items']
        )

        if artist:

            if len(artist) < max_iter:
                total_iter = len(artist)
            else:
                total_iter = max_iter

            for i in range(total_iter):

                if name == artist[i]['name']:
                    artist_dict.append(get_artist_dict(artist[i]))
                    break

            if len(artist_dict) == total_artist:
                no_data_artist.append(name)  
        else:
            no_data_artist.append(name)

    sp_artist = pd.DataFrame(artist_dict)

    genres = set(sum(sp_artist.genres.tolist(), []))

    metal_genres = [
        g for g in genres
        if any(m in g for m in config['metal_genres'])
    ]

    others = [g for g in genres if g not in metal_genres]

    sp_artist['is_metal_artist'] = sp_artist['genres'].apply(is_metal_artist)
    sp_artist.to_csv(f'data/refined/spotify_{l}.csv', index=False)

S data:  69%|██████▉   | 6140/8837 [39:18<17:27,  2.58it/s]  